# Linear Regression Code Along

This notebook is the reference for the video lecture on the Linear Regression Code Along. Basically what we do here is examine a dataset with Ecommerce Customer Data for a company's website and mobile app. Then we want to see if we can build a regression model that will predict the customer's yearly spend on the company's product.

## Sample Data

```
Email,Address,Avatar,Avg Session Length,Time on App,Time on Website,Length of Membership,Yearly Amount Spent

mstephenson@fernandez.com,"835 Frank TunnelWrightmouth, MI 82180-9605",Violet,34.49726772511229,12.65565114916675,39.57766801952616,4.0826206329529615,587.9510539684005

hduke@hotmail.com,"4547 Archer CommonDiazchester, CA 06566-8576",DarkGreen,31.92627202636016,11.109460728682564,37.268958868297744,2.66403418213262,392.2049334443264

pallen@yahoo.com,"24645 Valerie Unions Suite 582Cobbborough, DC 99414-7564",Bisque,33.000914755642675,11.330278057777512,37.110597442120856,4.104543202376424,487.54750486747207
```

First thing to do is start a Spark Session

In [1]:
# Import Relative Libs: SparkSession, LinearRegression, VectorAssembler
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

# Create Spark Session
spark = SparkSession.builder.appName("linear_regression").getOrCreate()

In [2]:
dir_input_path = "./input_data/"
file_input_path = dir_input_path + "Ecommerce_Customers.csv"

## Load input corpus

In [3]:
df = spark.read.csv(file_input_path, inferSchema=True, header=True)

## Print the Schema of the DataFrame

In [4]:
df.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



## Show head 2 lines

In [5]:
df.head(2)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005),
 Row(Email='hduke@hotmail.com', Address='4547 Archer CommonDiazchester, CA 06566-8576', Avatar='DarkGreen', Avg Session Length=31.92627202636016, Time on App=11.109460728682564, Time on Website=37.268958868297744, Length of Membership=2.66403418213262, Yearly Amount Spent=392.2049334443264)]

## Print each item in the first line

In [6]:
for item in df.head():
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


## Setting Up DataFrame for Machine Learning 

In [7]:
# A few things we need to do before Spark can accept the data!
# It needs to be in the form of two columns
# ("label","features")

# Import VectorAssembler

In [8]:
df.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [9]:
col_features = ['Avg Session Length', 
                'Time on App', 
                'Time on Website', 
                'Length of Membership']
assembler = VectorAssembler(inputCols=col_features, outputCol="features")

In [10]:
df_assembler = assembler.transform(df)

In [11]:
df_assembler.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))

In [12]:
df_assembler.show(1)

+--------------------+--------------------+------+------------------+-----------------+-----------------+--------------------+-------------------+--------------------+
|               Email|             Address|Avatar|Avg Session Length|      Time on App|  Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+------+------------------+-----------------+-----------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|Violet| 34.49726772511229|12.65565114916675|39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
+--------------------+--------------------+------+------------------+-----------------+-----------------+--------------------+-------------------+--------------------+
only showing top 1 row



In [13]:
df_all = df_assembler.select(["features", "Yearly Amount Spent"])

## Split to training set and testing set

In [14]:
train_set, test_set = df_all.randomSplit([0.7, 0.3])

In [15]:
train_set.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                352|
|   mean| 499.47281346594986|
| stddev|  80.07463471787126|
|    min|  282.4712457199145|
|    max|  744.2218671047146|
+-------+-------------------+



In [16]:
test_set.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                148|
|   mean|  498.9364107383854|
| stddev|  77.74469956678585|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



## Create a Linear Regression Model object

In [17]:
lr = LinearRegression(featuresCol="features", labelCol="Yearly Amount Spent")

## Fit the model to the data

In [18]:
model = lr.fit(train_set)

## Print the coefficients and intercept for linear regression

In [19]:
print("coefficients: {}\nand intercept: {}".format(model.coefficients, model.intercept))

coefficients: [25.614961200030688,38.6781045074737,0.16784060791156336,61.57001222656741]
and intercept: -1037.069751485636


## Show residuals after evaluating testing set

In [20]:
test_result = model.evaluate(test_set)

In [21]:
test_result.residuals.show(3)

+-------------------+
|          residuals|
+-------------------+
|  9.856879935562347|
| -12.43446396072261|
|-17.564782702181787|
+-------------------+
only showing top 3 rows



## Print MSE, RMSE and r2 of training set and after evaluating testing set

In [22]:
print("Test-set : MSE: {}\nRMSE: {}\nr2: {}".format(test_result.meanSquaredError, 
                                                    test_result.rootMeanSquaredError,
                                                    test_result.r2))

Test-set : MSE: 101.90337988623023
RMSE: 10.09472039663458
r2: 0.983025718894069


In [23]:
train_result = model.evaluate(train_set)
print("Train-set : MSE: {}\nRMSE: {}\nr2: {}".format(train_result.meanSquaredError, 
                                                     train_result.rootMeanSquaredError,
                                                     train_result.r2))

Train-set : MSE: 97.21812157599706
RMSE: 9.859925028923753
r2: 0.9847947753813873


Excellent results! Let's see how you handle some more realistically modeled data in the Consulting Project!